In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_colwidth = 5000
import matplotlib
import nltk
import swifter
import collections
from wordcloud import WordCloud
from prettytable import PrettyTable
from tabulate import tabulate


In [ ]:
# # Read data file (2019&2020)
# data = pd.read_feather('../data/feather_files/data2019clean.feather')
# data = pd.concat((data,\
#         pd.read_feather('../data/feather_files/data2020clean.feather')))
# # Take test data out
# test = data.query("tclass == 'test'").reset_index(drop=True)
# data = data.query("tclass == 'political' or tclass=='nonpolitical'").reset_index(drop=True)
# data.head(1)

train = pd.read_feather('../data/feather_files/splits/train_50.feather')
valid = pd.read_feather('../data/feather_files/splits/valid_50.feather')
proto = pd.read_feather('../data/feather_files/proto_50.feather')
protoc = pd.read_feather('../data/feather_files/protoc_50.feather')
valid.sample(3)


In [3]:
import logging.handlers
from datetime import date
# number of prototypical words to keep.
k = 1000
log_to_file = True


try:
    log.info("Log is already initiated.")
except:
    if log_to_file:
        logging.basicConfig(filename=f"../data/log/scores__.log",
                            filemode='a',
                            format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.DEBUG)
        log = logging.getLogger("Bot")
        log.info("###########################################################")

    else:
        log = logging.getLogger("bot")
        log.setLevel(logging.DEBUG)
        log.addHandler(logging.StreamHandler())

In [ ]:
#worddict = pd.read_feather('../data/feather_files/worddict_800.feather')
#wp_in_u = pd.read_feather('../data/feather_files/wp_in_u_800.feather')


In [4]:
def scorer(k):
    train = pd.read_feather(f'../data/feather_files/splits/train_{k}.feather')
    valid = pd.read_feather(f'../data/feather_files/splits/valid_{k}.feather')
    proto = pd.read_feather(f'../data/feather_files/proto_{k}.feather')
    protoc = pd.read_feather(f'../data/feather_files/protoc_{k}.feather')
    
    protom = pd.DataFrame()
    
    protom['sc_p'] = proto.apply(lambda d: sum(d.mul(protoc.sc_p)))
    protom['sc_np'] = proto.apply(lambda d: sum(d.mul(protoc.sc_np)))
    protom['pclass'] = protom.apply(lambda d: 'p' if d.sc_p > d.sc_np else 'np',axis=1)
    protom['word'] = proto.columns
    pprotoc = pd.DataFrame()
    # The numerator .swifter.allow_dask_on_strings()
    pprotoc['sc_p'] = valid.text.map(lambda d :  protom[protom.word.isin(d.split())].sc_p.sum())
    pprotoc['sc_np'] = valid.text.map(lambda d : protom[protom.word.isin(d.split())].sc_np.sum())
    pprotoc['tclass'] = valid.tclass
    protoc['tclass'] = train.tclass
    def predictor(d):
        if d.sc_p == d.sc_np == 0:
            return 0
        elif (d.sc_p > d.sc_np and d.tclass == 'political') or (d.sc_p < d.sc_np and d.tclass == 'nonpolitical'):
            return 1
        else:
            return -1

    protoc['preds'] = protoc.swifter.apply(predictor, axis=1)
    pprotoc['preds'] = pprotoc.swifter.apply(predictor, axis=1)


    accuracy = pd.DataFrame()
    acc = pprotoc.preds.value_counts()
    acc["Accuracy"] = str(np.round(acc[1]/(acc[1]+acc[-1]),4)*100) + '%'
    acc["non-Response"] = str(np.round(acc[0] / (acc[1]+acc[-1]+acc[0]),4)*100) + '%'
    accuracy[f'Valid_{k}'] = acc
    #-------------------------------------------------------------------------------
    acc = protoc.preds.value_counts()
    acc["Accuracy"] = str(np.round(acc[1]/(acc[1]+acc[-1]),4)*100) + '%'
    acc["non-Response"] = str(np.round(acc[0] / (acc[1]+acc[-1]+acc[0]),4)*100) + '%'
    accuracy[f'train_{k}'] = acc
    #----------------------------------------------------------------------
    return accuracy


In [5]:
 accuracy = scorer(1000)
# accuracy = pd.concat((accuracy,scorer(400)),axis=1)
# accuracy = pd.concat((accuracy,scorer(600)),axis=1)
# accuracy = pd.concat((accuracy,scorer(800)),axis=1)
# accuracy = pd.concat((accuracy,scorer(800)),axis=1)
# accuracy = pd.concat((accuracy,scorer(1000)),axis=1)

#----
print(tabulate(accuracy, headers='keys', tablefmt='psql'))
log.info(tabulate(accuracy, headers='keys', tablefmt='psql'))

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

+--------------+--------------+--------------+
|              | Valid_1000   | train_1000   |
|--------------+--------------+--------------|
| 1            | 76043        | 310326       |
| 0            | 16204        | 40055        |
| -1           | 8889         | 54159        |
| Accuracy     | 89.53%       | 85.14%       |
| non-Response | 16.02%       | 9.9%         |
+--------------+--------------+--------------+


In [ ]:
print(accuracy)

In [ ]:
k=200

In [ ]:
accuracy

In [ ]:
train.tclass.value_counts() / train.shape[0]
valid.tclass.value_counts() / valid.shape[0]

In [ ]:
%%time
accuracy = scorer(200)


In [ ]:
log.info(accuracy)

In [ ]:
%%time
print("8")


In [ ]:
accuracy


In [ ]:
# train = pd.read_feather(f'../data/feather_files/splits/train_{k}.feather')
# valid = pd.read_feather(f'../data/feather_files/splits/valid_{k}.feather')
# proto = pd.read_feather(f'../data/feather_files/proto_{k}.feather')
# protoc = pd.read_feather(f'../data/feather_files/protoc_{k}.feather')

# protom = pd.DataFrame()

# protom['sc_p'] = proto.apply(lambda d: sum(d.mul(protoc.sc_p)))
# protom['sc_np'] = proto.apply(lambda d: sum(d.mul(protoc.sc_np)))
# protom['pclass'] = protom.apply(lambda d: 'p' if d.sc_p > d.sc_np else 'np',axis=1)
# protom['word'] = proto.columns
# pprotoc = pd.DataFrame()
# # The numerator .swifter.allow_dask_on_strings()
# pprotoc['sc_p'], pprotoc['sc_np'] = valid.text\
# .apply(lambda d : protom[protom.word.isin(d.split())][['sc_p','sc_np']].sum(axis=0))
# #pprotoc['sc_np'] = valid.text.swifter.allow_dask_on_strings().apply(lambda d : protom[protom.word.isin(d.split())].sc_np.sum())
# pprotoc['tclass'] = valid.tclass
# protoc['tclass'] = train.tclass

# def predictor(d):
#     if d.sc_p == d.sc_np == 0:
#         return 0
#     elif (d.sc_p > d.sc_np and d.tclass == 'political') or (d.sc_p < d.sc_np and d.tclass == 'nonpolitical'):
#         return 1
#     else:
#         return -1

# protoc['preds'] = protoc.swifter.apply(predictor, axis=1)
# pprotoc['preds'] = pprotoc.swifter.apply(predictor, axis=1)


accuracy = pd.DataFrame()
acc = pprotoc.preds.value_counts()
acc["Accuracy"] = str(np.round(acc[1]/(acc[1]+acc[-1]),4)*100) + '%'
acc["non-Response"] = str(np.round(acc[0] / (acc[1]+acc[-1]+acc[0]),4)*100) + '%'
accuracy[f'Valid_{k}'] = acc
#-------------------------------------------------------------------------------
acc = protoc.preds.value_counts()
acc["Accuracy"] = str(np.round(acc[1]/(acc[1]+acc[-1]),4)*100) + '%'
acc["non-Response"] = str(np.round(acc[0] / (acc[1]+acc[-1]+acc[0]),4)*100) + '%'
accuracy[f'train_{k}'] = acc

In [ ]:
pprotoc

In [ ]:
protom

In [ ]:
PrettyTable(accuracy)

In [ ]:

table = PrettyTable()

table.title = 'Most common 15 words in each category'
table.add_column('Political', np.transpose(counter_p.most_common(15))[0])
table.add_column('Non-Political', np.transpose(counter_np.most_common(15))[0])
table.add_column('Canada', np.transpose(counter_t.most_common(15))[0])
log.info(table)
log.info(table.get_latex_string())